In [3]:
import argparse
import logging
import json
import numpy as np
import os
import trimesh
from pathos.multiprocessing import ProcessPool as Pool
from scipy.spatial import cKDTree as KDTree
from scipy.optimize import linear_sum_assignment


In [4]:
#From DeepSDF metrics 
# https://github.com/facebookresearch/DeepSDF/tree/main/deep_sdf/metrics
# 

def compute_trimesh_chamfer(gt_points, gen_mesh, offset, scale, num_mesh_samples=30000):
    """
    This function computes a symmetric chamfer distance, i.e. the sum of both chamfers.

    gt_points: trimesh.points.PointCloud of just poins, sampled from the surface (see
               compute_metrics.ply for more documentation)

    gen_mesh: trimesh.base.Trimesh of output mesh from whichever autoencoding reconstruction
              method (see compute_metrics.py for more)

    """

    gen_points_sampled = trimesh.sample.sample_surface(gen_mesh, num_mesh_samples)[0]
   # print(gen_points_sampled)
    gen_points_sampled = gen_points_sampled / scale - offset

    # only need numpy array of points
    # gt_points_np = gt_points.vertices
   # print(gt_points)
    gt_points_np = gt_points.vertices

    # one direction
    gen_points_kd_tree = KDTree(gen_points_sampled)
    one_distances, one_vertex_ids = gen_points_kd_tree.query(gt_points_np)
    gt_to_gen_chamfer = np.mean(np.square(one_distances))

    # other direction
    gt_points_kd_tree = KDTree(gt_points_np)
    two_distances, two_vertex_ids = gt_points_kd_tree.query(gen_points_sampled)
    gen_to_gt_chamfer = np.mean(np.square(two_distances))

    return gt_to_gen_chamfer + gen_to_gt_chamfer

In [5]:
# From DIT
#https://github.com/ZhengZerong/DeepImplicitTemplates/tree/master/deep_sdf/metrics

def compute_trimesh_emd(gt_points, gen_mesh, offset, scale, num_mesh_samples=30000):
    """
    This function computes a symmetric chamfer distance, i.e. the sum of both chamfers.

    gt_points: trimesh.points.PointCloud of just poins, sampled from the surface (see
               compute_metrics.ply for more documentation)

    gen_mesh: trimesh.base.Trimesh of output mesh from whichever autoencoding reconstruction
              method (see compute_metrics.py for more)

    """

    gen_points_sampled = trimesh.sample.sample_surface(gen_mesh, num_mesh_samples)[0]

    gen_points_sampled = gen_points_sampled / scale - offset

    # only need numpy array of points
    # gt_points_np = gt_points.vertices
    gt_points_np = gt_points.vertices
    gt_points_np = np.random.permutation(gt_points_np)[:num_mesh_samples]

    # hist0 = hist1 = np.ones([num_mesh_samples], dtype=np.float64) / num_mesh_samples
    dist = np.linalg.norm(np.expand_dims(gt_points_np, axis=0) - np.expand_dims(gen_points_sampled, axis=1), axis=-1)
    # dist = dist.astype(np.float64)
    # emd = pyemd.emd(hist0, hist1, dist)
    assignment = linear_sum_assignment(dist)
    emd = dist[assignment].sum() / num_mesh_samples

    return emd


In [13]:
DIRECTORY_MODELS = './reconstruction/04256520/mesh' # 04256520: Sofa
#DIRECTORY_MODELS = './reconstruction/02691156/mesh' # 02691156 : Plane
#DIRECTORY_MODELS = './reconstruction/04379243/mesh' # 04379243: Table

MODEL_EXTENSION = '.ply'
file_name = []
file_list = []
for directory, _,files in os.walk(DIRECTORY_MODELS):
 #   print(files)
    for filename in files:
        if filename.endswith(MODEL_EXTENSION):

            file_name.append(filename[:-4])
            file_list.append(os.path.join(filename))
                

print("the number of Dataset:", len(file_list))
gt_dir = 'gt/'
if not os.path.exists(gt_dir):
    os.mkdir(gt_dir)

cl1 = DIRECTORY_MODELS.split('/')[2]
    
DIRECTORY_GT = gt_dir + cl1
GT_EXTENSION = '.obj'
gt_obj = []
for i in range(len(file_name)):
#    if 
    dir = DIRECTORY_GT + "/" + file_name[i] +"/"
  #  print(file_name[i])
    if os.path.exists(dir):
        a = 1
    else:
        continue
        
    for directory, _, gt_files in os.walk(dir):
        for gt_file in gt_files:
            if gt_file.endswith(GT_EXTENSION):
                gt_obj.append(os.path.join(directory+ '/' + gt_file))
                
                
                
# print(len(gt_obj))
# #print(sorted(file_list))

the number of Dataset: 3


In [14]:
DIRECTORY_GT

'gt/04256520'

In [171]:
cd_mean = []
emd_mean = []
#print(file_list)
for i in range(len(file_list)):
#    if 
    print(file_list[i])
    rec_data = trimesh.load(DIRECTORY_MODELS +"/"+ file_list[i])
    dir = DIRECTORY_GT + "/" + file_name[i] +"/"
  #  print(file_name[i])
    if os.path.exists(dir):
        a = 1
    else:
        continue
        
    for directory, _, gt_files in os.walk(dir):
        for gt_file in gt_files:
            if gt_file.endswith(GT_EXTENSION):
                gt_obj = (os.path.join(directory+ '/' + gt_file))
                gt_data =trimesh.load(gt_obj, force='mesh')
    Chamfer = compute_trimesh_chamfer(rec_data,gt_data,0, 2)
    EMD = compute_trimesh_emd(rec_data,gt_data,0, 2)
   #print("EMD", EMD, "Chamfer", Chamfer)
    cd_mean.append(Chamfer)
    emd_mean.append(EMD)
CD_M = sum(cd_mean)/len(cd_mean)
EMD_M = sum(emd_mean)/len(emd_mean)
cd_m = np.median(cd_mean)
emd_m = np.median(emd_mean)
print("Chamfer: ",CD_M, "Median: ",cd_m)
print("EMD: ", EMD_M, "Median: ", emd_m)




122963149f6a04272620819ddac40644.ply


concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts


10eeb119fd5508e0d6d949577c389a84.ply
121b5c1c81aa77906b153e6e0582b3ac.ply


concatenating texture: may result in visual artifacts


117830993cc5887726587cb13c78fb9b.ply
10cfc2090a2ade124c3a35cee92bb95b.ply
103c9e43cdf6501c62b600da24e0965.ply
10e4331c34d610dacc14f1e6f4f4f49b.ply
1203825bf97bc3524722e1824a086fad.ply


...

121e9fceb90440efed79d3bd546890bd.ply


concatenating texture: may result in visual artifacts


122776d17b6a118086da73d36506db6f.ply


concatenating texture: may result in visual artifacts


10c7cdfdffe2243b88a89a28f04ce622.ply
110f6dbf0e6216e9f9a63e9a8c332e52.ply
10db820f0e20396a492c7ca609cb0182.ply
10aa040f470500c6a66ef8df4909ded9.ply
Chamfer:  0.1514755198514165 Median:  0.1554586844755423
EMD:  0.39621568200779106 Median:  0.40751884032050933


In [133]:
os.path.exists("../gt/04356520/4b86c8d86e181ed8f51f77a6d7299806/models/")

True

In [ ]:



for ii, data in enumerate(file_list):
    
    data_dir = os.path.join(DIRECTORY_MODELS, data)
    rec_data = trimesh.load(data_dir)
   # print("data_dir: ", data_dir)
    gt_dir = os.path.join(DIRECTORY_GT, data[:-4], "models/")
  #  print(gt_dir)
  #  print("os.path.exists(gt_dir): ",os.path.exists(gt_dir))
    if os.path.exists(gt_dir):
        a = 1
      #  print("YES")
    else:
        continue
        
    for directory, _, gt_files in os.walk(gt_dir):
        for gt_file in gt_files:
            if gt_file.endswith(GT_EXTENSION):
                gt_obj = (os.path.join(directory+ '/' + gt_file))
                print(gt_obj)
                gt_data =trimesh.load(gt_obj, force='mesh')
    Chamfer = compute_trimesh_chamfer(rec_data,gt_data,0, 2)
    EMD = compute_trimesh_emd(rec_data,gt_data,0, 2)
   #print("EMD", EMD, "Chamfer", Chamfer)
    cd_mean.append(Chamfer)
    emd_mean.append(EMD)
CD_M = sum(cd_mean)/len(cd_mean)
EMD_M = sum(emd_mean)/len(emd_mean)
cd_m = np.median(cd_mean)
emd_m = np.median(emd_mean)
print("Chamfer: ",CD_M, "Median: ",cd_m)
print("EMD: ", EMD_M, "Median: ", emd_m)